In [1]:
import programs.new_builder as nb
from pmagpy import ipmag
import os
import json
import numpy as np
import sys

import pandas as pd
from pandas import DataFrame
from pmagpy import builder
from pmagpy import validate_upload
from pmagpy import pmag

reload(nb)

pmagpy_dir = os.path.join(os.getcwd(), 'pmagpy')
if pmagpy_dir not in sys.path:
    sys.path.append(pmagpy_dir)
    
WD = os.path.join(os.getcwd(), "3_0", "McMurdo")


In [2]:
# get specimen level data from specimen table
contribution = nb.Contribution(WD, read_tables=['measurements', 'specimens', 'samples','sites'])
if 'specimens' in contribution.tables:
    spec_container = contribution.tables['specimens']
    spec_data = spec_container.df
spec_data.columns
spec_data.loc[['mc121d1', 'mc01a']][['aniso_s']]

,aniso_s
specimen,
mc121d1,0.29925 : 0.346846 : 0.353904 : -0.001806 : -0...
mc121d1,None
mc121d1,None
mc01a,None
mc01a,None


In [3]:
# find the record of a particular specimen.  if it already has aniso data, then modify in place
# otherwise add a new record and modify that....    
specimen = 'mc121d1' # a specimen with aniso data
new_s = '.3:.3:.3:0:0:0'

# add numeric index column temporarily
spec_data['num'] = range(len(spec_data))

# iterate through specimen names and update or add anisotropy data
for spec in [specimen, "mc01a"]:
    # match specimen name
    cond1 = spec_data['specimen'].str.contains(spec) == True
    # anisotropy data is present
    cond2 = spec_data['aniso_s'].notnull() == True
    cond = (cond1 & cond2)
    if len(spec_data[cond]) > 0:
        print 'update existing record', spec
        # get list of all rows for which both conditions are true
        inds = spec_data[cond]['num']
        print inds
        # in case there are multiple anisotropy rows for a specimen, update each row
        for ind in inds:
            # get pre-existing data from that row
            existing_data = dict(spec_data.iloc[ind])
            # add to existing_data with values from new_data
            new_data = {'aniso_s': new_s, 'specimen': spec}
            existing_data.update(new_data)
            # update row
            spec_container.update_row(ind, existing_data)
    else:
        print 'no record found - creating new one', spec
        # add new row
        spec_container.add_row(spec, {'aniso_s': new_s, 'specimen': spec})


# sort so that all rows for a specimen are together
spec_data.sort_index(inplace=True)
# redo temporary index
spec_data['num'] = range(len(spec_data))

# see all records for mc01a & mc121d1 
# now includes 1 new anisotropy record for mc01a, and 1 updated record for mc121d1
spec_data.loc[['mc01a', specimen]][['specimen', 'aniso_s', 'num']]

update existing record mc121d1
specimen
mc121d1    282
Name: num, dtype: int64
no record found - creating new one mc01a


,specimen,aniso_s,num
specimen,,,
mc01a,mc01a,None,0
mc01a,mc01a,.3:.3:.3:0:0:0,1
mc01a,mc01a,None,2
mc121d1,mc121d1,None,283
mc121d1,mc121d1,.3:.3:.3:0:0:0,284
mc121d1,mc121d1,None,285


In [4]:
## original way:
#spec_data['aniso_s']=np.where(spec_data['specimen'].str.contains(specimen) & spec_data['aniso_s'].notnull(), new_s,spec_data['aniso_s'])
#spec_data['specimen'].str.contains(specimen) & spec_data['aniso_s'].notnull()

In [3]:
def flip_dict(dict):
    newdict={}
    for key in dict:
        val=dict[key]
        newdict[val]=key
    return newdict

In [4]:
spec3_spec2={'int_drats': 'specimen_drats', 'site': 'er_site_name', 'int_mad': 'specimen_int_mad', 'sample': 'er_sample_name', 'measurement_step_max': 'meas_step_max', 'specimen_n': 'dir_n_measurements', 'int_n_measurements': 'specimen_int_n', 'int_corr': 'specimen_correction', 'int_rsc': 'specimen_rsc', 'analyst_names': 'er_analyst_mail_names', 'int_scat': 'specimen_scat', 'int_ptrm_n': 'specimen_int_ptrm_n', 'citations': 'er_citation_names', 'int_gmax': 'specimen_gmax', 'int_dang': 'specimen_int_dang', 'dir_tilt_correction': 'specimen_tilt_correction', 'location': 'er_location_name', 'dir_comp': 'specimen_comp_name', 'specimen_magn_moment': 'magn_moment', 'int_w': 'specimen_w', 'specimen': 'er_specimen_name', 'int_q': 'specimen_q', 'int_fvds': 'specimen_fvds', 'specimen_mad': 'dir_mad_free', 'int_frac': 'specimen_frac', 'meas_step_min': 'measurement_step_min', 'int_f': 'specimen_f', 'software_packages': 'magic_software_packages', 'dir_mad_free': 'specimen_mad', 'magn_moment': 'specimen_magn_moment', 'instrument_codes': 'magic_instrument_codes', 'int_b_beta': 'specimen_b_beta', 'dir_n_comps': 'specimen_comp_n', 'int_md': 'specimen_md', 'dir_n_measurements': 'specimen_n', 'dir_inc': 'specimen_inc', 'specimen_magn_volumn': 'magn_volumn', 'meas_step_max': 'measurement_step_max', 'dir_alpha95': 'specimen_alpha95', 'magn_volumne': 'specimen_magn_volumn', 'measurement_step_min': 'meas_step_min', 'meas_step_unit': 'measurement_step_unit', 'dir_dec': 'specimen_dec', 'method_codes': 'magic_method_codes', 'result_quality': 'specimen_flag', 'dir_dang': 'specimen_dang'} 
site3_site2={'int_abs_sigma' : 'site_int_sigma', 'int_abs_sigma_perc' : 'site_int_sigma_perc', 'int_n_samples' : 'site_int_n', 'dir_alpha95' : 'site_alpha95', 'dir_k' : 'site_k', 'dir_n_samples' : 'site_n', 'dir_n_specimens_lines' : 'site_n_lines', 'dir_n_specimens_planes' : 'site_n_planes', 'dir_r' : 'site_r'}
aniso3_aniso2={'specimen':'er_specimen_name', 'aniso_type':'anisotropy_type', 'description':'result_description', 'aniso_ftest':'anisotropy_ftest', 'aniso_ftest12':'anisotropy_ftest12', 'aniso_ftest23':'anisotropy_ftest23', 'aniso_s_mean':'anisotropy_mean', 'aniso_s_n_measurements':'anisotropy_n', 'aniso_s_sigma':'anisotropy_sigma', 'aniso_s_unit':'anisotropy_unit', 'aniso_tilt_correction':'anisotropy_tilt_correction'}




print flip_dict(aniso3_aniso2)


{'anisotropy_ftest23': 'aniso_ftest23', 'anisotropy_ftest': 'aniso_ftest', 'anisotropy_sigma': 'aniso_s_sigma', 'anisotropy_type': 'aniso_type', 'anisotropy_ftest12': 'aniso_ftest12', 'anisotropy_tilt_correction': 'aniso_tilt_correction', 'er_specimen_name': 'specimen', 'anisotropy_unit': 'aniso_s_unit', 'anisotropy_mean': 'aniso_s_mean', 'result_description': 'description', 'anisotropy_n': 'aniso_s_n_measurements'}
